In [2]:
import requests
import json
import pandas as pd

In [ ]:
url='https://python.zach.lol'

In [ ]:
print(requests.get(url).json())

In [ ]:
response = requests.get(url)

In [ ]:
response.json()

In [ ]:
print(requests.get(url+'/documentation').json()['payload'])

In [ ]:
response = requests.get(url+'/api/v1/items')

In [ ]:
print(response.json()['payload']['next_page'])
print(response.json()['payload']['max_page'])

In [ ]:
items = [pd.DataFrame(response.json()['payload']['items'])]

In [ ]:
response = requests.get(url+response.json()['payload']['next_page'])

In [ ]:
items.append(pd.DataFrame(response.json()['payload']['items']))

In [ ]:
response = requests.get(url+response.json()['payload']['next_page'])

In [ ]:
items.append(pd.DataFrame(response.json()['payload']['items']))

In [ ]:
print(response.json()['payload']['next_page'])

In [ ]:
items = pd.concat(items).reset_index().drop(columns="index")

In [8]:
def get_items():
    url='https://python.zach.lol'
    response = requests.get(url+'/api/v1/items')
    items = [pd.DataFrame(response.json()['payload']['items'])]
    response = requests.get(url+response.json()['payload']['next_page'])
    items.append(pd.DataFrame(response.json()['payload']['items']))
    response = requests.get(url+response.json()['payload']['next_page'])
    items.append(pd.DataFrame(response.json()['payload']['items']))
    print(response.json()['payload']['next_page'])
    return pd.concat(items).reset_index().drop(columns="index")

In [32]:
items = get_items()

None


## Stores

In [ ]:
print(requests.get(url+'/documentation').json()['payload'])

In [ ]:
response = requests.get(url+'/api/v1/stores').json()

In [ ]:
print(f"max page: {response['payload']['max_page']}") 
print(f"next page: {response['payload']['next_page']}")

In [ ]:
response['payload'].keys()

In [ ]:
stores = pd.DataFrame(response['payload']['stores'])

In [5]:
def get_stores():
    url='https://python.zach.lol'
    response = requests.get(url+'/api/v1/stores').json()
    print(f"max page: {response['payload']['max_page']}") 
    print(f"next page: {response['payload']['next_page']}")
    return pd.DataFrame(response['payload']['stores'])

In [33]:
stores = get_stores()

max page: 1
next page: None


## Sales

In [ ]:
print(requests.get(url+'/documentation').json()['payload'])

In [ ]:
response = requests.get(url+'/api/v1/sales').json()

In [ ]:
print(f"payload keys: {response['payload'].keys()}")
print(f"max_page: {response['payload']['max_page']}")

In [ ]:
requests.get(url+response['payload']['next_page']).json()

In [ ]:
final_page = '/api/v1/sales?page=183'

In [ ]:
final_page = requests.get(url+final_page).json()

In [27]:
def get_sales():   
    url='https://python.zach.lol'
    counter = 0
    sales= []
    response = requests.get(url+'/api/v1/sales').json()
    while True:
        sales.append(pd.DataFrame(response['payload']['sales']))
        counter += 1
        if counter%10==0:
            print(f"Pages {counter-9} - {counter} loaded successfully")
        if response['payload']['next_page'] == None:
            print('No next page, acquisition complete')
            print(f"{counter} pages loaded successfully")
            break
        response = requests.get(url+response['payload']['next_page']).json()
    print('merging pages into DataFrame')
    return pd.concat(sales).reset_index().drop(columns="index")

In [28]:
sales = get_sales()

Pages 1 - 10 loaded successfully
Pages 11 - 20 loaded successfully
Pages 21 - 30 loaded successfully
Pages 31 - 40 loaded successfully
Pages 41 - 50 loaded successfully
Pages 51 - 60 loaded successfully
Pages 61 - 70 loaded successfully
Pages 71 - 80 loaded successfully
Pages 81 - 90 loaded successfully
Pages 91 - 100 loaded successfully
Pages 101 - 110 loaded successfully
Pages 111 - 120 loaded successfully
Pages 121 - 130 loaded successfully
Pages 131 - 140 loaded successfully
Pages 141 - 150 loaded successfully
Pages 151 - 160 loaded successfully
Pages 161 - 170 loaded successfully
Pages 171 - 180 loaded successfully
No next page, acquisition complete
183 pages loaded successfully
merging pages into DataFrame


In [31]:
sales.tail()

,item,sale_amount,sale_date,sale_id,store
912995,50,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",912996,10
912996,50,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",912997,10
912997,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",912998,10
912998,50,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",912999,10
912999,50,82.0,"Sun, 31 Dec 2017 00:00:00 GMT",913000,10


In [36]:
items.tail()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
45,Mama Marys,46,Pizza Sauce,4.65,35457770664,35457770664
46,Bear Naked,47,Bear Naked Fit Almond Crisp 100 Percent Natura...,7.38,884623708976,884623708976
47,Dove,48,Dove Men + Care Antiperspirant Deodorant Cool ...,3.72,79400271631,79400271631
48,Easy-off,49,Easy-off Oven Cleaner Lemon Scent,9.54,62338879772,62338879772
49,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221


In [37]:
stores.tail()

,store_address,store_city,store_id,store_state,store_zipcode
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


In [42]:
sales = sales.rename({'store':"store_id",'item':'item_id'},axis=1)

In [53]:
merged = pd.merge(sales,items)

In [56]:
merged = pd.merge(merged,stores)

In [57]:
merged.shape

(913000, 14)

In [58]:
merged.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
